In [3]:
import pandas as pd

# TEST NOTEBOOK TO IMPLEMENT THE TKAN MODEL
"""
from model.TKAN import Model
a = Model()
a.compile_execute_test()
"""

'\nfrom model.TKAN import Model\na = Model()\na.compile_execute_test()\n'

In [28]:
import DataExtraction.meteocat as meteocat
import DataExtraction.utils as utils
import requests
import pandas as pd
measured = meteocat.__MEASURED_DATA_URL
# meteocat.print_api_usage()
print(measured)

['https://api.meteo.cat/xema/v1/variables/mesurades/', '/', '/', '/', '?codiEstacio=']


In [10]:
varcode ='35'
year = '2024'
month = '11'
day = '12'

url = measured[0] + varcode + measured[1] + year + measured[2] + month + measured[3] + day + measured[4]
url

'https://api.meteo.cat/xema/v1/variables/mesurades/35/2024/11/12?codiEstacio='

In [11]:
response = requests.get(url, headers=meteocat.__HEADERS)

In [12]:
response.json()

[{'codi': 'C6',
  'variables': [{'codi': 35,
    'lectures': [{'data': '2024-11-12T00:00Z',
      'valor': 0,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2024-11-12T00:30Z',
      'valor': 0,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2024-11-12T01:00Z',
      'valor': 0,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2024-11-12T01:30Z',
      'valor': 0,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2024-11-12T02:00Z',
      'valor': 0,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2024-11-12T02:30Z',
      'valor': 0,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2024-11-12T03:00Z',
      'valor': 0,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2024-11-12T03:30Z',
      'valor': 0,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2024-11-12T04:00Z',
      'valor': 0,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2024-11-12T04:30Z',

In [35]:
df = pd.json_normalize(response.json(),  record_path=['variables', 'lectures'], meta=['codi', ['variables', 'codi']])

In [36]:
df['data'] = df['data'].apply(lambda x: utils.parse_date(x, input_format="%Y-%m-%dT%H:%MZ"))
df['data'] = df['data'].dt.date

In [39]:
df.drop(columns=['estat', 'baseHoraria', 'variables.codi'], inplace=True)

In [41]:
df_final_today = df.groupby(['data', 'codi']).sum().reset_index()

,data,codi,valor
0,2024-11-12,C6,0.1
1,2024-11-12,C7,0.1
2,2024-11-12,C8,0.0
3,2024-11-12,C9,7.9
4,2024-11-12,CC,0.0
...,...,...,...
180,2024-11-12,Z7,0.3
181,2024-11-12,Z9,0.0
182,2024-11-12,ZB,0.0
183,2024-11-12,ZC,0.0


In [7]:
import DataExtraction.meteocat as meteocat
a = meteocat.add_today_information('35')
b = meteocat.transform_daily_data(meteocat.get_daily_data('1300', '2024', '11'))

1300
2024
11
Calling meteocat_raw API
Retrieving information: Daily data from all stations - Year: 2024 Month: 11 - Var: 1300
Response status code: 200 - OK


In [8]:
print(a.shape)
print(b.shape)

(185, 4)
(2960, 4)


In [9]:
c = meteocat.join_daily_and_today_data(b, a)

In [10]:
c.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3145 entries, 0 to 184
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   data          3145 non-null   object 
 1   valor         3145 non-null   float64
 2   codiEstacio   3145 non-null   object 
 3   codiVariable  3145 non-null   object 
dtypes: float64(1), object(3)
memory usage: 122.9+ KB


In [12]:
c['codiVariable'].unique()

array([1300], dtype=object)